# [COM4513-6513] Assignment 1: Text Classification with Logistic Regression

### Instructor: Nikos Aletras


The goal of this assignment is to develop and test two text classification systems:

- **Task 1:** sentiment analysis, in particular to predict the sentiment of movie review, i.e. positive or negative (binary classification).
- **Task 2:** topic classification, to predict whether a news article is about International issues, Sports or Business (multiclass classification).


For that purpose, you will implement:

- Text processing methods for extracting Bag-Of-Word features, using (1) unigrams, bigrams and trigrams to obtain vector representations of documents. Two vector weighting schemes should be tested: (1) raw frequencies (**3 marks; 1 for each ngram type**); (2) tf.idf (**1 marks**).
- Binary Logistic Regression classifiers that will be able to accurately classify movie reviews trained with (1) BOW-count (raw frequencies); and (2) BOW-tfidf (tf.idf weighted) for Task 1.
- Multiclass Logistic Regression classifiers that will be able to accurately classify news articles trained with (1) BOW-count (raw frequencies); and (2) BOW-tfidf (tf.idf weighted) for Task 2.
- The Stochastic Gradient Descent (SGD) algorithm to estimate the parameters of your Logistic Regression models. Your SGD algorithm should:
    - Minimise the Binary Cross-entropy loss function for Task 1 (**3 marks**)
    - Minimise the Categorical Cross-entropy loss function for Task 2 (**3 marks**)
    - Use L2 regularisation (both tasks) (**1 mark**)
    - Perform multiple passes (epochs) over the training data (**1 mark**)
    - Randomise the order of training data after each pass (**1 mark**)
    - Stop training if the difference between the current and previous validation loss is smaller than a threshold (**1 mark**)
    - After each epoch print the training and development loss (**1 mark**)
- Discuss how did you choose hyperparameters (e.g. learning rate and regularisation strength)?  (**2 marks; 0.5 for each model in each task**).
- After training the LR models, plot the learning process (i.e. training and validation loss in each epoch) using a line plot (**1 mark; 0.5 for both BOW-count and BOW-tfidf LR models in each task**) and discuss if your model overfits/underfits/is about right.
- Model interpretability by showing the most important features for each class (i.e. most positive/negative weights). Give the top 10 for each class and comment on whether they make sense (if they don't you might have a bug!).  If we were to apply the classifier we've learned into a different domain such laptop reviews or restaurant reviews, do you think these features would generalise well? Can you propose what features the classifier could pick up as important in the new domain? (**2 marks; 0.5 for BOW-count and BOW-tfidf LR models respectively in each task**)


### Data - Task 1

The data you will use for Task 1 are taken from here: [http://www.cs.cornell.edu/people/pabo/movie-review-data/](http://www.cs.cornell.edu/people/pabo/movie-review-data/) and you can find it in the `./data_sentiment` folder in CSV format:

- `data_sentiment/train.csv`: contains 1,400 reviews, 700 positive (label: 1) and 700 negative (label: 0) to be used for training.
- `data_sentiment/dev.csv`: contains 200 reviews, 100 positive and 100 negative to be used for hyperparameter selection and monitoring the training process.
- `data_sentiment/test.csv`: contains 400 reviews, 200 positive and 200 negative to be used for testing.

### Data - Task 2

The data you will use for Task 2 is a subset of the [AG News Corpus](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html) and you can find it in the `./data_topic` folder in CSV format:

- `data_topic/train.csv`: contains 2,400 news articles, 800 for each class to be used for training.
- `data_topic/dev.csv`: contains 150 news articles, 50 for each class to be used for hyperparameter selection and monitoring the training process.
- `data_topic/test.csv`: contains 900 news articles, 300 for each class to be used for testing.


### Submission Instructions

You should submit a Jupyter Notebook file (assignment1.ipynb) and an exported PDF version (you can do it from Jupyter: `File->Download as->PDF via Latex`).

You are advised to follow the code structure given in this notebook by completing all given funtions. You can also write any auxilliary/helper functions (and arguments for the functions) that you might need but note that you can provide a full solution without any such functions. Similarly, you can just use only the packages imported below but you are free to use any functionality from the [Python Standard Library](https://docs.python.org/2/library/index.html), NumPy, SciPy and Pandas. You are not allowed to use any third-party library such as Scikit-learn (apart from metric functions already provided), NLTK, Spacy, Keras etc..

Please make sure to comment your code. You should also mention if you've used Windows (not recommended) to write and test your code. There is no single correct answer on what your accuracy should be, but correct implementations usually achieve F1-scores around 80\% or higher. The quality of the analysis of the results is as important as the accuracy itself.

This assignment will be marked out of 20. It is worth 20\% of your final grade in the module.

The deadline for this assignment is **23:59 on Fri, 20 Mar 2020** and it needs to be submitted via MOLE. Standard departmental penalties for lateness will be applied. We use a range of strategies to detect [unfair means](https://www.sheffield.ac.uk/ssid/unfair-means/index), including Turnitin which helps detect plagiarism, so make sure you do not plagiarise.



In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

# fixing random seed for reproducibility
random.seed(123)
np.random.seed(123)

## Load Raw texts and labels into arrays

First, you need to load the training, development and test sets from their corresponding CSV files (tip: you can use Pandas dataframes).

In [ ]:
sentiment_dev = pd.read_csv('data_sentiment/dev.csv', names=['text', 'label'])
sentiment_test = pd.read_csv('data_sentiment/test.csv', names=['text', 'label'])
sentiment_train = pd.read_csv('data_sentiment/train.csv', names=['text', 'label'])

If you use Pandas you can see a sample of the data.

In [ ]:
sentiment_train.head()

The next step is to put the raw texts into Python lists and their corresponding labels into NumPy arrays:


In [ ]:
sentiment_dev_texts = list(sentiment_dev['text'])
sentiment_dev_labels = np.array(sentiment_dev['label'])

sentiment_test_texts = list(sentiment_test['text'])
sentiment_test_labels = np.array(sentiment_test['label'])

sentiment_train_texts = list(sentiment_train['text'])
sentiment_train_labels = np.array(sentiment_train['label'])

# Bag-of-Words Representation


To train and test Logisitc Regression models, you first need to obtain vector representations for all documents given a vocabulary of features (unigrams, bigrams, trigrams).


## Text Pre-Processing Pipeline

To obtain a vocabulary of features, you should:
- tokenise all texts into a list of unigrams (tip: using a regular expression)
- remove stop words (using the one provided or one of your preference)
- compute bigrams, trigrams given the remaining unigrams
- remove ngrams appearing in less than K documents
- use the remaining to create a vocabulary of unigrams, bigrams and trigrams (you can keep top N if you encounter memory issues).


In [ ]:
default_stop_words = {
    'a', 'all', 'am', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'by', 'can',
    'could', 'did', 'do', 'eg', 'etc', 'for', 'from', 'has', 'have', 'he',
    'her', 'hers', 'herself', 'him', 'himself', 'his', 'i', 'ie', 'in', 'inc',
    'is', 'it', 'its', 'itself', 'ltd', 'me', 'mine', 'my', 'myself', 'of',
    'on', 'or', 'our', 'ours', 'ourselves', 'own', 'she', 'spoiler', 'spoilers',
    'that', 'the', 'their', 'theirs', 'them', 'themselves', 'there', 'these',
    'they', 'this', 'those', 'to', 've', 'was', 'we', 'were', 'what', 'which',
    'who', 'will', 'with', 'you', 'your', 'yours', 'yourself', 'yourselves'
}

### N-gram extraction from a document

You first need to implement the `extract_ngrams` function. It takes as input:
- `x_raw`: a string corresponding to the raw text of a document
- `ngram_range`: a tuple of two integers denoting the type of ngrams you want to extract, e.g. (1,2) denotes extracting unigrams and bigrams.
- `token_pattern`: a string to be used within a regular expression to extract all tokens. Note that data is already tokenised so you could opt for a simple white space tokenisation.
- `stop_words`: a list of stop words
- `vocab`: a given vocabulary. It should be used to extract specific features.

and returns:

- a list of all extracted features.

See the examples below to see how this function should work.

In [ ]:
def extract_ngrams(x_raw,
                   ngram_range=(1, 3),
                   token_pattern=r'\b[A-Za-z]{2,}\b',
                   stop_words=default_stop_words,
                   vocab=None):

    if vocab is None:
        vocab = set()

    tokens = [
        word for word in re.findall(token_pattern, x_raw)
        if word not in stop_words
    ]

    ngrams = []

    for n in range(ngram_range[0], ngram_range[1] + 1):
        if n == 1:
            # Create unigram by concatenating list
            ngrams += tokens
        else:
            # Create bigram / trigram by unzipping list
            ngrams += zip(*(tokens[i:] for i in range(n)))

    return list(set(ngrams) & vocab) if vocab else ngrams

In [ ]:
extract_ngrams('this is a great movie to watch')

In [ ]:
extract_ngrams('this is a great movie to watch',
               ngram_range=(1, 2),
               vocab={'great', ('great', 'movie')})

Note that it is OK to represent n-grams using lists instead of tuples: e.g. `['great', ['great', 'movie']]`

### Create a vocabulary of n-grams

Then the `get_vocab` function will be used to (1) create a vocabulary of ngrams; (2) count the document frequencies of ngrams; (3) their raw frequency. It takes as input:
- `X_raw`: a list of strings each corresponding to the raw text of a document
- `ngram_range`: a tuple of two integers denoting the type of ngrams you want to extract, e.g. (1,2) denotes extracting unigrams and bigrams.
- `token_pattern`: a string to be used within a regular expression to extract all tokens. Note that data is already tokenised so you could opt for a simple white space tokenisation.
- `stop_words`: a list of stop words
- `min_df`: keep ngrams with a minimum document frequency.
- `keep_topN`: keep top-N more frequent ngrams.

and returns:

- `vocab`: a set of the n-grams that will be used as features.
- `df`: a Counter (or dict) that contains ngrams as keys and their corresponding document frequency as values.
- `ngram_counts`: counts of each ngram in vocab

Hint: it should make use of the `extract_ngrams` function.

In [ ]:
def get_vocab(X_raw,
              ngram_range=(1, 3),
              token_pattern=r'\b[A-Za-z]{2,}\b',
              min_df=0,
              keep_topN=None,
              stop_words=default_stop_words):

    df = Counter()
    ngram_counts = Counter()

    # A list containing each document's ngrams as sublist, e.g. [ [ngram, ngram, ...], ..., ]
    ngram_lists = (extract_ngrams(text, ngram_range, token_pattern, stop_words) for text in X_raw)

    for ngram_list in ngram_lists:
        # Count document frequency
        df.update(set(ngram_list))

        # Count ngram frequency
        ngram_counts.update(ngram for ngram in ngram_list if df[ngram] >= min_df)

    # Extract ngram into vocab set
    vocab = {ngram for ngram, _ in ngram_counts.most_common(keep_topN)}

    return vocab, df, ngram_counts

Now you should use `get_vocab` to create your vocabulary and get document and raw frequencies of n-grams:

In [ ]:
vocab, df, ngram_counts = get_vocab(sentiment_train_texts, keep_topN=5000)
print(len(vocab))
print()
print(list(vocab)[:100])
print()
print(df.most_common()[:10])

Then, you need to create vocabulary id -> word and id -> word dictionaries for reference:

In [ ]:
id_to_word = enumerate(vocab)
id_to_word = dict(id_to_word)


word_to_id = {value: key for key, value in id_to_word.items()}


Now you should be able to extract n-grams for each text in the training, development and test sets:

In [ ]:
sentiment_train_vocab, sentiment_train_df, sentiment_train_ngram_counts = get_vocab(
    sentiment_train_texts, keep_topN=5000)

sentiment_dev_vocab, sentiment_dev_df, sentiment_dev_ngram_counts = get_vocab(
    sentiment_dev_texts, keep_topN=5000)

sentiment_test_vocab, sentiment_test_df, sentiment_test_ngram_counts = get_vocab(
    sentiment_test_texts, keep_topN=5000)

## Vectorise documents

Next, write a function `vectoriser` to obtain Bag-of-ngram representations for a list of documents. The function should take as input:
- `X_ngram`: a list of texts (documents), where each text is represented as list of n-grams in the `vocab`
- `vocab`: a set of n-grams to be used for representing the documents

and return:
- `X_vec`: an array with dimensionality Nx|vocab| where N is the number of documents and |vocab| is the size of the vocabulary. Each element of the array should represent the frequency of a given n-gram in a document.


In [ ]:
def vectorise(X_ngram, vocab):
    X_vec = []

    for ngram_list in X_ngram:
        counter = Counter(ngram_list)
        X_vec.append([counter[v] for v in vocab])

    X_vec = np.array(X_vec)

    return X_vec

Finally, use `vectorise` to obtain document vectors for each document in the train, development and test set. You should extract both count and tf.idf vectors respectively:

#### Count vectors

In [ ]:
sentiment_train_count = vectorise(
    (extract_ngrams(text) for text in sentiment_train_texts),
    sentiment_train_vocab)

sentiment_dev_count = vectorise(
    (extract_ngrams(text) for text in sentiment_dev_texts),
    sentiment_train_vocab)

sentiment_test_count = vectorise(
    (extract_ngrams(text) for text in sentiment_test_texts),
    sentiment_train_vocab)

In [ ]:
sentiment_train_count.shape

In [ ]:
sentiment_train_count[:2,:50]

#### TF.IDF vectors

First compute `idfs` an array containing inverted document frequencies (Note: its elements should correspond to your `vocab`)

In [ ]:
total_sentiment_train_docs = len(sentiment_train_texts)
total_sentiment_dev_docs = len(sentiment_dev_texts)
total_sentiment_test_docs = len(sentiment_test_texts)

sentiment_train_idf = np.array([np.log(total_sentiment_train_docs / sentiment_train_ngram_counts[ngram])
                                for ngram in sentiment_train_vocab])

sentiment_dev_idf = np.array([np.log(total_sentiment_dev_docs / sentiment_dev_ngram_counts[ngram])
                              for ngram in sentiment_dev_vocab])

sentiment_test_idf = np.array([np.log(total_sentiment_test_docs / sentiment_test_ngram_counts[ngram])
                              for ngram in sentiment_test_vocab])

Then transform your count vectors to tf.idf vectors:

In [ ]:
sentiment_train_tfidf = sentiment_train_count * sentiment_train_idf

sentiment_dev_tfidf = sentiment_dev_count * sentiment_dev_idf

sentiment_test_tfidf = sentiment_test_count * sentiment_test_idf

In [ ]:
sentiment_train_tfidf[1, :50]

# Binary Logistic Regression

After obtaining vector representations of the data, now you are ready to implement Binary Logistic Regression for classifying sentiment.

First, you need to implement the `sigmoid` function. It takes as input:

- `z`: a real number or an array of real numbers

and returns:

- `sig`: the sigmoid of `z`

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
print(sigmoid(0))
print(sigmoid(np.array([-5., 1.2])))

Then, implement the `predict_proba` function to obtain prediction probabilities. It takes as input:

- `X`: an array of inputs, i.e. documents represented by bag-of-ngram vectors ($N \times |vocab|$)
- `weights`: a 1-D array of the model's weights $(1, |vocab|)$

and returns:

- `preds_proba`: the prediction probabilities of X given the weights

In [ ]:
def predict_proba(X, weights):

    z = X.dot(weights)

    return sigmoid(z)

Then, implement the `predict_class` function to obtain the most probable class for each vector in an array of input vectors. It takes as input:

- `X`: an array of documents represented by bag-of-ngram vectors ($N \times |vocab|$)
- `weights`: a 1-D array of the model's weights $(1, |vocab|)$

and returns:

- `preds_class`: the predicted class for each x in X given the weights

In [ ]:
def predict_class(X, weights):

    probabilities = predict_proba(X, weights)

    return np.array([0 if prob < 0.5 else 1 for prob in probabilities])

To learn the weights from data, we need to minimise the binary cross-entropy loss. Implement `binary_loss` that takes as input:

- `X`: input vectors
- `Y`: labels
- `weights`: model weights
- `alpha`: regularisation strength

and return:

- `l`: the loss score

In [ ]:
def binary_loss(X, Y, weights, alpha=0.00001):
    predicted_probabilities = predict_proba(X, weights)

    l = -Y * np.log(predicted_probabilities) - (1 - Y) * np.log(1 - predicted_probabilities)

    # L2 Regularisation
    l += alpha * weights.dot(weights)

    return l

Now, you can implement Stochastic Gradient Descent to learn the weights of your sentiment classifier. The `SGD` function takes as input:

- `X_tr`: array of training data (vectors)
- `Y_tr`: labels of `X_tr`
- `X_dev`: array of development (i.e. validation) data (vectors)
- `Y_dev`: labels of `X_dev`
- `lr`: learning rate
- `alpha`: regularisation strength
- `epochs`: number of full passes over the training data
- `tolerance`: stop training if the difference between the current and previous validation loss is smaller than a threshold
- `print_progress`: flag for printing the training progress (train/validation loss)


and returns:

- `weights`: the weights learned
- `training_loss_history`: an array with the average losses of the whole training set after each epoch
- `validation_loss_history`: an array with the average losses of the whole development set after each epoch

In [ ]:
def SGD(X_tr, Y_tr, X_dev, Y_dev, lr=0.1, alpha=0.00001, epochs=5, tolerance=0.0001, print_progress=True):

    training_loss_history = []
    validation_loss_history = []

    weights = np.zeros(X_tr.shape[1])

    D_tr = list(zip(X_tr, Y_tr))

    for epoch in range(epochs):
        # Randomise order in D_tr
        np.random.shuffle(D_tr)

        for x_i, y_i in D_tr:
            weights -= lr * ((predict_proba(x_i, weights) - y_i) * x_i - 2 * alpha * weights)

        # Monitor training and validation loss
        cur_loss_tr = np.mean(binary_loss(X_tr, Y_tr, weights, alpha))
        cur_loss_dev = np.mean(binary_loss(X_dev, Y_dev, weights, alpha))

        # Early stopping
        if epoch > 0 and validation_loss_history[-1] - cur_loss_dev < tolerance:
            break
        else:
            training_loss_history.append(cur_loss_tr)
            validation_loss_history.append(cur_loss_dev)

        if print_progress:
            print(f"Epoch: {epoch} | Training loss: {cur_loss_tr} | Validation loss: {cur_loss_dev}")
            
    return weights, training_loss_history, validation_loss_history

## Train and Evaluate Logistic Regression with Count vectors

First train the model using SGD:

In [ ]:
w_count, tr_loss_count, dev_loss_count = SGD(X_tr=sentiment_train_count,
                                             Y_tr=sentiment_train_labels,
                                             X_dev=sentiment_dev_count,
                                             Y_dev=sentiment_dev_labels,
                                             lr=0.0001,
                                             alpha=0.001,
                                             epochs=100)

Now plot the training and validation history per epoch. Does your model underfit, overfit or is it about right? Explain why.

In [ ]:
plt.plot(tr_loss_count, label='training loss')
plt.plot(dev_loss_count, label='validation loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.title('Training Monitoring')

plt.legend()

plt.show()

Explain here...

Compute accuracy, precision, recall and F1-scores:

In [ ]:
preds_test_count = predict_class(sentiment_test_count, w_count)

print('Accuracy:', accuracy_score(sentiment_test_labels, preds_test_count))
print('Precision:', precision_score(sentiment_test_labels, preds_test_count))
print('Recall:', recall_score(sentiment_test_labels, preds_test_count))
print('F1-Score:', f1_score(sentiment_test_labels, preds_test_count))

Finally, print the top-10 words for the negative and positive class respectively.

In [ ]:
# fill in your code...

If we were to apply the classifier we've learned into a different domain such laptop reviews or restaurant reviews, do you think these features would generalise well? Can you propose what features the classifier could pick up as important in the new domain?

Provide your answer here...

## Train and Evaluate Logistic Regression with TF.IDF vectors

Follow the same steps as above (i.e. evaluating count n-gram representations).


In [ ]:
w_tfidf, trl, devl = SGD(X_tr_tfidf, Y_tr,
                         X_dev=X_dev_tfidf,
                         Y_dev=Y_dev,
                         lr=0.0001,
                         alpha=0.00001,
                         epochs=50)

Now plot the training and validation history per epoch. Does your model underfit, overfit or is it about right? Explain why.

In [ ]:
# fill in your code...

Compute accuracy, precision, recall and F1-scores:

In [ ]:
# fill in your code...

print('Accuracy:', accuracy_score(Y_te,preds_te))
print('Precision:', precision_score(Y_te,preds_te))
print('Recall:', recall_score(Y_te,preds_te))
print('F1-Score:', f1_score(Y_te,preds_te))

Print top-10 most positive and negative words:

In [ ]:
# fill in your code...

In [ ]:
# fill in your code...

### Discuss how did you choose model hyperparameters (e.g. learning rate and regularisation strength)? What is the relation between training epochs and learning rate? How the regularisation strength affects performance?

Enter your answer here...



## Full Results

Add here your results:

| LR | Precision  | Recall  | F1-Score  |
|:-:|:-:|:-:|:-:|
| BOW-count  |   |   |   |
| BOW-tfidf  |   |   |   |


# Multi-class Logistic Regression

Now you need to train a Multiclass Logistic Regression (MLR) Classifier by extending the Binary model you developed above. You will use the MLR model to perform topic classification on the AG news dataset consisting of three classes:

- Class 1: World
- Class 2: Sports
- Class 3: Business

You need to follow the same process as in Task 1 for data processing and feature extraction by reusing the functions you wrote.

In [ ]:
topic_dev = pd.read_csv('data_topic/dev.csv', names=['label', 'text'])
topic_test = pd.read_csv('data_topic/test.csv', names=['label', 'text'])
topic_train = pd.read_csv('data_topic/train.csv', names=['label', 'text'])

In [ ]:
topic_train.head()

In [ ]:
topic_dev_texts = list(topic_dev['text'])
topic_dev_labels = np.array(topic_dev['label'])

topic_test_texts = list(topic_test['text'])
label_test_labels = np.array(topic_test['label'])

topic_train_texts = list(topic_train['text'])
topic_train_labels = np.array(topic_train['label'])

In [ ]:
vocab, df, ngram_counts = get_vocab(X_tr_raw, ngram_range=(1,3), keep_topN=5000)
print(len(vocab))
print()
print(list(vocab)[:100])
print()
print(df.most_common()[:10])

In [ ]:
# fill in your code...

Now you need to change `SGD` to support multiclass datasets. First you need to develop a `softmax` function. It takes as input:

- `z`: array of real numbers

and returns:

- `smax`: the softmax of `z`

In [ ]:
def softmax(z):

    # fill in your code...

    return smax

Then modify `predict_proba` and `predict_class` functions for the multiclass case:

In [ ]:
def predict_proba(X, weights):

    # fill in your code...

    return preds_proba

In [ ]:
def predict_class(X, weights):

    # fill in your code...

    return preds_class

Toy example and expected functionality of the functions above:

In [ ]:
X = np.array([[0.1,0.2],[0.2,0.1],[0.1,-0.2]])
w = np.array([[2,-5],[-5,2]])

In [ ]:
predict_proba(X, w)

In [ ]:
predict_class(X, w)

Now you need to compute the categorical cross entropy loss (extending the binary loss to support multiple classes).

In [ ]:
def categorical_loss(X, Y, weights, num_classes=5, alpha=0.00001):

    # fill in your code...

    return l


Finally you need to modify SGD to support the categorical cross entropy loss:

In [ ]:
def SGD(X_tr, Y_tr, X_dev=[], Y_dev=[], num_classes=5, lr=0.01, alpha=0.00001, epochs=5, tolerance=0.001, print_progress=True):

    # fill in your code...

    return weights, training_loss_history, validation_loss_history

Now you are ready to train and evaluate you MLR following the same steps as in Task 1 for both Count and tfidf features:

In [ ]:
w_count, loss_tr_count, dev_loss_count = SGD(X_tr_count, Y_tr,
                                             X_dev=X_dev_count,
                                             Y_dev=Y_dev,
                                             num_classes=3,
                                             lr=0.0001,
                                             alpha=0.001,
                                             epochs=200)

Plot training and validation process and explain if your model overfit, underfit or is about right:

In [ ]:
# fill in your code...

Compute accuracy, precision, recall and F1-scores:

In [ ]:
# fill in your code...

print('Accuracy:', accuracy_score(Y_te,preds_te))
print('Precision:', precision_score(Y_te,preds_te,average='macro'))
print('Recall:', recall_score(Y_te,preds_te,average='macro'))
print('F1-Score:', f1_score(Y_te,preds_te,average='macro'))

Print the top-10 words for each class respectively.

In [ ]:
# fill in your code...

### Discuss how did you choose model hyperparameters (e.g. learning rate and regularisation strength)? What is the relation between training epochs and learning rate? How the regularisation strength affects performance?

Explain here...

### Now evaluate BOW-tfidf...

## Full Results

Add here your results:

| LR | Precision  | Recall  | F1-Score  |
|:-:|:-:|:-:|:-:|
| BOW-count  |   |   |   |
| BOW-tfidf  |   |   |   |
